In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from pyESN.pyESN import ESN
from utils import *
from scipy.signal import periodogram,decimate
# rng = np.random.RandomState(42)

In [ ]:
metadata = read_metadata_by_fileindex(1)
labels = metadata['label']

metadata = metadata.iloc[:,2:]
ctrl = np.hstack([np.ones((len(metadata),1)),metadata.iloc[:,0].to_numpy().reshape(-1,1)])
output = labels.to_numpy().reshape(len(labels),1)
ctrl = ctrl.astype(float)
output = output.astype(float)

In [ ]:
N = len(ctrl) # signal length
traintest_cutoff = int(np.ceil(0.7*N))

In [ ]:
train_ctrl,train_output = ctrl[:traintest_cutoff],output[:traintest_cutoff]
test_ctrl, test_output  = ctrl[traintest_cutoff:],output[traintest_cutoff:]

In [135]:
esn = ESN(n_inputs = 2,
          n_outputs = 1,
          n_reservoir = 5000,
          spectral_radius = .25,
          sparsity = 0.95,
          noise = 0.001,
          input_shift = [0,0],
          input_scaling = [0.01, 3],
          teacher_scaling = 1.12,
          teacher_shift = -0.7,
          out_activation = np.tanh,
          inverse_out_activation = np.arctanh,
          silent = False)

pred_train,W = esn.fit(train_ctrl,train_output)
print("test error:")
pred_test = esn.predict(test_ctrl,test_output)
print(np.sqrt(np.mean((pred_test - test_output)**2)))

harvesting states...
fitting...
training error:
0.06426631978272594
test error:


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
plt.figure(figsize=(10,1.5))
# plt.plot(train_ctrl[:,1],label='control')
plt.plot(train_output,label='target')
plt.plot(pred_train,label='model')
plt.legend(fontsize='x-small')
plt.title('training (excerpt)')

plt.figure(figsize=(10,4))
# plt.plot(test_ctrl[:,1],label='control')
plt.plot(test_output,label='target')
plt.plot(pred_test,label='model')
plt.legend(fontsize='x-small')
plt.title('test (excerpt)')


In [ ]:
df = pd.DataFrame(W)
ws = []
df = df[df!=0]
for col in df.columns:
    for weight in df[col].dropna().to_numpy():
        ws.append(weight)
plt.hist(ws)